In [ ]:
import pandas as pd  # For data handling (creating and manipulating dataframes)
import matplotlib.pyplot as plt  # For data visualization (creating plots)
import seaborn as sns  # For statistical visualization (creating attractive plots)
import geopandas as gpd  # For geospatial analysis (working with maps and location-based data)
import networkx as nx  # For graph-based route optimization (finding shortest paths)
import numpy as np  # For numerical computations (mathematical operations)
from prophet import Prophet  # For time series forecasting (predicting future trends)
from sklearn.ensemble import IsolationForest  # For anomaly detection (fraud detection)
from sklearn.cluster import KMeans  # For customer segmentation (clustering)
from sklearn.linear_model import LogisticRegression  # For churn prediction (classification)
from scipy.optimize import linprog  # For optimization (cost reduction)

In [ ]:
# Load Data
df = pd.read_csv("/content/fedex.csv")  # Load dataset from CSV file


In [ ]:
# Display first few rows
print("Initial Data Preview:\n", df.head())  # Print the first 5 rows to understand the data structure

In [ ]:
# Rename Columns for better readability
df.rename(columns={'old_col1': 'new_col1', 'old_col2': 'new_col2'}, inplace=True)  # Rename columns for clarity

In [ ]:
print(df.columns)  # Print the exact column names to verify themprint(df.columns)  # Yeh batayega ke columns ka exact naam kya hai


In [ ]:
# Ensure Year, Month, and DayofMonth are integers and handle missing values
df['Year'] = df['Year'].astype(int)  # Convert 'Year' column to integer
df['Month'] = df['Month'].astype(int)  # Convert 'Month' column to integer
df['DayofMonth'] = df['DayofMonth'].fillna(1).astype(int)  # Replace missing values with 1 and convert to integer

In [ ]:
# Convert Year, Month, DayofMonth into a proper Date column using apply()
df['Date'] = pd.to_datetime(df.apply(lambda row: f"{row['Year']}-{row['Month']:02d}-{row['DayofMonth']:02d}", axis=1), errors='coerce')  # Create a 'Date' column by combining Year, Month, and DayofMonth# Convert Year, Month, DayofMonth into a proper Date column using apply()
df['Date'] = pd.to_datetime(df.apply(lambda row: f"{row['Year']}-{row['Month']:02d}-{row['DayofMonth']:02d}", axis=1), errors='coerce')

In [ ]:
# Extract Month in 'YYYY-MM' format for monthly trend analysis
df['Month'] = df['Date'].dt.to_period('M')  # Convert 'Date' to 'YYYY-MM' format for monthly analysis

In [ ]:
# Check final dataframe
print(df[['Year', 'Month', 'DayofMonth', 'Date']].head())  # Print the first 5 rows of selected columns to verify the transformations

In [ ]:
# Monthly Shipment Delay Trend Analysis

# Step 1: Set the figure size for the plot
plt.figure(figsize=(12, 6))  # Create a figure with a specific size (12x6 inches)

# Step 2: Group shipment delay by month and calculate the total delay per month
monthly_delay = df.groupby('Month')['Shipment_Delay'].sum().reset_index()  # Group by 'Month' and sum 'Shipment_Delay'

# Step 3: Convert 'Shipment_Delay' to numeric (in case it contains non-numeric values)
monthly_delay['Shipment_Delay'] = pd.to_numeric(monthly_delay['Shipment_Delay'], errors='coerce')  # Convert to numeric, coercing errors to NaN

# Step 4: Handle missing values in 'Shipment_Delay' (replace NaN with 0)
monthly_delay['Shipment_Delay'] = monthly_delay['Shipment_Delay'].fillna(0)  # Fill missing values with 0

# Step 5: Ensure 'Month' is in a proper format (convert to string if needed)
monthly_delay['Month'] = monthly_delay['Month'].astype(str)  # Convert 'Month' to string for proper plotting

# Step 6: Plot the shipment delay trend using Seaborn
sns.lineplot(data=monthly_delay, x='Month', y='Shipment_Delay')  # Create a line plot for monthly delay trend

# Step 7: Set plot title and format x-axis
plt.title("Month-on-Month Shipment Delay Trend")  # Add a title to the plot
plt.xticks(rotation=45)  # Rotate x-axis labels by 45 degrees for better readability
plt.xlabel("Month")  # Label for the x-axis
plt.ylabel("Total Shipment Delay")  # Label for the y-axis

# Step 8: Display the plot
plt.show()  # Show the plot

In [ ]:
# Top Routes Analysis
# Step 1: Check column names to verify the correct column for counting orders
print("Column Names in DataFrame:\n", df.columns)  # Print all column names in the DataFrame

# Step 2: Group by Source and Destination, and count the number of orders per route
# Replace 'Order_ID' with the correct column name (e.g., 'OrderID', 'OrderNumber', etc.)
# If no order-specific column exists, use the row count as a proxy
if 'Order_ID' in df.columns:
    top_routes = df.groupby(['Source', 'Destination'])['Order_ID'].count().reset_index()  # Count orders per route
    top_routes = top_routes.rename(columns={'Order_ID': 'OrderCount'})  # Rename the count column for clarity
else:
    top_routes = df.groupby(['Source', 'Destination']).size().reset_index(name='OrderCount')  # Use row count as a proxy

# Step 3: Sort the routes by the number of orders in descending order and select the top 10
top_routes = top_routes.sort_values(by='OrderCount', ascending=False).head(10)  # Select top 10 routes

# Step 4: Display the top 10 most frequent routes
print("Top 10 Most Frequent Routes:\n", top_routes)  # Print the top 10 routes

# Optimal Route Suggestion using Efficiency Score
# Step 1: Calculate the Efficiency Score (Distance / Time)
# Ensure 'Distance' and 'Time' columns exist and are numeric
if 'Distance' in df.columns and 'Time' in df.columns:
    df['Efficiency_Score'] = df['Distance'] / df['Time']  # Lower score means better efficiency

    # Step 2: Find the route with the minimum Efficiency Score (most efficient route)
    best_route = df.loc[df['Efficiency_Score'].idxmin(), ['Source', 'Destination']]  # Find the most efficient route

    # Step 3: Display the optimal route
    print("Optimal Route:", best_route)  # Print the most efficient route
else:
    print("Columns 'Distance' or 'Time' not found. Cannot calculate Efficiency Score.")

# Sales Seasonality Analysis
# Step 1: Ensure 'Date' and 'Sales' columns exist
if 'Date' in df.columns and 'Sales' in df.columns:
    # Step 2: Set the figure size for the plot
    plt.figure(figsize=(12, 6))  # Create a figure with a specific size (12x6 inches)

    # Step 3: Create a boxplot to visualize sales seasonality by month
    sns.boxplot(x=df['Date'].dt.month, y=df['Sales'])  # x-axis: Month, y-axis: Sales

    # Step 4: Add a title to the plot
    plt.title("Sales Seasonality by Month")  # Set the title of the plot

    # Step 5: Display the plot
    plt.show()  # Show the plot
else:
    print("Columns 'Date' or 'Sales' not found. Cannot perform Sales Seasonality Analysis.")

In [ ]:
# Customer Segmentation using K-Means Clustering

# Step 1: Check if 'Customer_ID', 'Sales', and 'Order_ID' columns exist
if 'Customer_ID' in df.columns and 'Sales' in df.columns and 'Order_ID' in df.columns:
    # Step 2: Aggregate sales and order count per customer
    customer_data = df.groupby('Customer_ID').agg({'Sales': 'sum', 'Order_ID': 'count'}).reset_index()  # Aggregate sales and orders per customer

    # Step 3: Check if the aggregated data has valid values
    if not customer_data.empty:
        # Step 4: Define the number of clusters (e.g., 3 customer segments)
        kmeans = KMeans(n_clusters=3, random_state=42)  # Initialize K-Means with 3 clusters

        # Step 5: Apply K-Means clustering to the aggregated data
        customer_data['Cluster'] = kmeans.fit_predict(customer_data[['Sales', 'Order_ID']])  # Assign clusters

        # Step 6: Visualize the customer segments using a scatter plot
        plt.figure(figsize=(10, 6))  # Set the figure size
        sns.scatterplot(data=customer_data, x='Sales', y='Order_ID', hue='Cluster', palette='viridis')  # Create scatter plot
        plt.title("Customer Segmentation")  # Add a title
        plt.xlabel("Total Sales")  # Label for x-axis
        plt.ylabel("Number of Orders")  # Label for y-axis
        plt.show()  # Display the plot
    else:
        print("No data available for clustering. Please check the input data.")
else:
    print("Required columns ('Customer_ID', 'Sales', or 'Order_ID') not found. Cannot perform customer segmentation.")

In [ ]:
df['Shipment_Delay_Days'] = df['Shipment_Delay'] / 1440  # Convert minutes to days
sns.histplot(df['Shipment_Delay_Days'], bins=30, kde=True, color='blue')
plt.xlabel("Shipment Delay in Days")
plt.title("Shipment Delay Distribution")
plt.show()


In [ ]:
late_deliveries = df[df['Shipment_Delay'] > 0]
print(f"Total Late Deliveries: {len(late_deliveries)}")
df['Delivery_Status_Label'] = df['Shipment_Delay'].apply(lambda x: 'Late' if x > 0 else 'On-Time')
sns.countplot(data=df, x='Delivery_Status_Label', palette='coolwarm')
plt.title("Late vs On-Time Deliveries")
plt.show()


In [ ]:
# Sentiment Analysis Placeholder (For Customer Reviews)
if 'Review_Text' in df.columns:
    print("Sentiment Analysis can be applied on Review_Text column")  # Check if sentiment analysis is possible